In [1]:
import pandas as pd
from bqplot import ColorScale, ColorAxis, DateScale, LinearScale, Axis, Lines, Figure
from ipywidgets import Label, VBox, Layout
from bqplot.market_map import MarketMap
import os

## Get Data

In [2]:
data = pd.read_csv(os.path.abspath('../data_files/country_codes.csv'), index_col=[0])
country_codes = data.index.values
country_names = data['Name']

## Basic Market Map

In [3]:
market_map = MarketMap(names=country_codes,      
                       # basic data which needs to set for each map
                       ref_data=data,            
                       # Data frame which can be used for different properties of the map
                       # Axis and scale for color data
                       tooltip_fields=['Name'])  

market_map

/Users/scorlay/jupyter/bquant/bqplot/bqplot/traits.py:202: UserWarning: The 'ref_data' dataframe trait of the <class 'bqplot.market_map.MarketMap'> instance disregards the index name
  warnings.warn("The '%s' dataframe trait of the %s instance disregards the index name" % (trait.name, trait.this_class))


In [4]:
market_map.colors = ['MediumSeaGreen']

In [5]:
market_map.font_style = {'font-size': '22px', 'fill':'white'}

In [6]:
market_map.title = 'My First Market Map'
market_map.title_style = {'fill': 'Red'}

## GDP data with grouping by continent

World Bank national accounts data, and OECD National Accounts data files. (The World Bank: GDP per capita (current US$))

In [7]:
gdp_data = pd.read_csv(os.path.abspath('../data_files/gdp_per_capita.csv'), index_col=[0], parse_dates=True)
gdp_data.fillna(method='backfill', inplace=True)
gdp_data.fillna(method='ffill', inplace=True)

In [8]:
col = ColorScale(scheme='Greens')
continents = data['Continent'].values
ax_c = ColorAxis(scale=col, label='GDP per Capita', visible=False)

data['GDP'] = gdp_data.ix[-1]

market_map = MarketMap(names=country_codes, groups=continents,       # Basic data which needs to set for each map
                       cols=25, row_groups=3,                        # Properties for the visualization
                       ref_data=data,                                # Data frame used for different properties of the map
                       tooltip_fields=['Name', 'Continent', 'GDP'],  # Columns from data frame to be displayed as tooltip
                       tooltip_formats=['', '', '.1f'],
                       scales={'color': col}, axes=[ax_c])           # Axis and scale for color data

/Users/scorlay/jupyter/bquant/bqplot/bqplot/traits.py:202: UserWarning: The 'ref_data' dataframe trait of the <class 'bqplot.market_map.MarketMap'> instance disregards the index name
  warnings.warn("The '%s' dataframe trait of the %s instance disregards the index name" % (trait.name, trait.this_class))


In [9]:
deb_output = Label()
def selected_index_changed(change):
    deb_output.value = str(change.new)
        
market_map.observe(selected_index_changed, 'selected')
VBox([deb_output, market_map])

In [10]:
# Attribute to show the names of the groups, in this case the continents
market_map.show_groups = True

In [11]:
# Setting the selected countries
market_map.show_groups = False
market_map.selected = ['PAN', 'FRA', 'PHL']

In [12]:
# changing selected stroke and hovered stroke variable
market_map.selected_stroke = 'yellow'
market_map.hovered_stroke = 'violet'

## Setting the color based on data

In [13]:
# Adding data for color and making color axis visible
market_map.color = data['GDP']
ax_c.visible = True

## Adding a widget as tooltip

In [14]:
# Creating the figure to be displayed as the tooltip
sc_x = DateScale()
sc_y = LinearScale()

ax_x = Axis(scale=sc_x, grid_lines='dashed', label='Date')
ax_y = Axis(scale=sc_y, orientation='vertical', grid_lines='dashed',
            label='GDP', label_location='end', label_offset='-1em')

line = Lines(x= gdp_data.index.values, y=[], scales={'x': sc_x, 'y': sc_y}, colors=['orange'])
fig_tooltip = Figure(marks=[line], axes=[ax_x, ax_y], layout=Layout(min_width='600px'))

In [15]:
market_map = MarketMap(names=country_codes, groups=continents,
                       cols=25, row_groups=3,
                       color=data['GDP'], scales={'color': col}, axes=[ax_c],
                       ref_data=data, tooltip_widget=fig_tooltip)

# Update the tooltip chart
hovered_symbol = ''
def hover_handler(self, content):
    global hovered_symbol
    symbol = content.get('ref_data', {}).get('Country Code', '')
    if(symbol != hovered_symbol):
        hovered_symbol = symbol
        if(gdp_data.get(hovered_symbol) is not None):
            line.y = gdp_data[hovered_symbol].values
            fig_tooltip.title = content.get('ref_data', {}).get('Name', '')

# Custom msg sent when a particular cell is hovered on
market_map.on_hover(hover_handler)
market_map

/Users/scorlay/jupyter/bquant/bqplot/bqplot/traits.py:202: UserWarning: The 'ref_data' dataframe trait of the <class 'bqplot.market_map.MarketMap'> instance disregards the index name
  warnings.warn("The '%s' dataframe trait of the %s instance disregards the index name" % (trait.name, trait.this_class))


This notebook uses data derived from the World Bank dataset.
- The World Bank: GDP per capita (current US$)
- The World Bank: Country Codes 

See the LICENSE file for more information.